# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Задание - построить модель с максимально большим значением *accuracy*. Проект считается успешным, если доля правильных ответов больше 0.75. 

## Откройте и изучите файл

In [3]:
import pandas as pd
import matplotlib.pyplot as plt  
import seaborn as sns 
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [6]:
data = pd.read_csv('users_behavior.csv')
data

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Названия колонок:

сalls — количество звонков,  
minutes — суммарная длительность звонков в минутах,  
messages — количество sms-сообщений,  
mb_used — израсходованный интернет-трафик в Мб,  
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [ ]:
sns.pairplot(data, hue="is_ultra");

In [ ]:
data['is_ultra'].plot(kind='hist', bins=2, figsize=(2,4), grid=True);

Мы проверили данные на мультиколлениарность, теперь можем заметить, что высокая корреляция у параметров minutes и calls, следовательно, можно исключить один из этих параметров, чтобы избежать переообучения.  
Однако мы можем заметить, что данных по тарифу Смарт более чем в 2 раза больше, чем по тарифу ультра. Скорее всего, это приведёт к более низкому качеству модели относительно тарифа Ультра. Однако так как у нас задача на бинарную классификацию, то дисбаланс не приведёт к большим проблемам.

Обучать модели будем по столбцу is_ultra.

## Разбейте данные на выборки

In [ ]:
features = data.drop(['is_ultra', 'calls'], axis=1)
target = data['is_ultra']

features_train, features_v, target_train, target_v = train_test_split(
    features, target, test_size=.4, random_state=1, stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_v, target_v, test_size=.4, random_state=1, stratify=target_v)


print(f"Количество строк в features_train по классам: {np.bincount(target_train)} {features_train.shape[0]/data.shape[0]:1.1} выборки")
print(f"Количество строк в features_valid по классам: {np.bincount(target_valid)} {features_valid.shape[0]/data.shape[0]:1.1} выборки")
print(f"Количество строк в features_test по классам: {np.bincount(target_test)} {features_test.shape[0]/data.shape[0]:1.1} выборки")


Так, мы получили 3 выборки:  
<pre>
features_train (данные), target_train (результаты) - тренировочная выборка  
features_valid (данные), target_valid (результаты) - валидационная выборка
features_test  (данные), target_test  (результаты) - тестовая выборка
    </pre>
Также сохранили баланс классов (2.26/1) и проверили размерность выборок 3:1:1.

## Исследуйте модели

### Логистическая регрессия

In [ ]:
#log_model = LogisticRegression(solver='lbfgs') #0.68
#log_model = LogisticRegression(solver='newton-cg', multi_class='ovr') #0.73
log_model = LogisticRegression(solver='newton-cg', multi_class='multinomial') #0.73
#log_model = LogisticRegression(solver='newton-cg', multi_class='auto') #0.73
#log_model = LogisticRegression(solver='liblinear') #0.72
#log_model = LogisticRegression(solver='sag') #0.67
#log_model = LogisticRegression(solver='saga') #0.67
log_model.fit(features_train, target_train)
log_predictions_valid = log_model.predict(features_valid)
print(accuracy_score(target_valid, log_predictions_valid ), 
      (mean_squared_error(target_valid, log_predictions_valid))**.5 )

Анализ параметров и гиперпараметров:    
- для данной выборки больше всего подходит метод newton-cg 
- параметр multi_class не влияет на качество модели при методе newton-cg

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(target_valid, log_predictions_valid)).plot();

Матрица показывает, что у нас 520+61 верных прогнозов и 175+15 неверных прогнозов. Посмотрим на другие метрики:

In [ ]:
print(classification_report(target_valid, log_predictions_valid))

precision - процент правильных положительных прогнозов по отношению к общему количеству положительных прогнозов.  
recall - процент правильных положительных прогнозов по отношению к общему количеству фактических положительных результатов.  
f1-score - средневзвешенное гармоническое значение точности и полноты. Чем ближе к 1, тем лучше модель.  

Точность предсказаний для тарифа Смарт равна 85%, точность предсказаний тарифа Ультра 39%.

### Дерево решений

In [ ]:
tree_best_model = None
tree_best_result = 0
tree_best_depth = 0
tree_predictions = None
model_plot_v = []
model_plot_i = []
for depth in range(1, 50):
    tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree_model.fit(features_train, target_train) 
    tree_predictions_valid = tree_model.predict(features_valid) 
    result = accuracy_score(target_valid, tree_predictions_valid) 
    if result > tree_best_result:
        tree_predictions = tree_predictions_valid
        tree_best_model = tree_model
        tree_best_result = result
        tree_best_depth = depth
    model_plot_v.append(result)
    model_plot_i.append(depth)

print("Accuracy наилучшей модели на валидационной выборке:", tree_best_result, "Глубина дерева:", tree_best_depth)
pd.Series(model_plot_v, index=model_plot_i).plot(grid=True)
plt.title('Успешность модели дерева решений в зависимости от глубины')
plt.xlabel("Глубина")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(target_valid, tree_predictions)).plot();

Матрица показывает, что у нас 502+118 верных прогнозов и 33+118 неверных прогнозов.

In [ ]:
print(classification_report(target_valid, tree_predictions))

Точность предсказаний для тарифа Смарт равна 87%, точность предсказаний тарифа Ультра 61%.

Аccuracy больше 0.75, а RMSE близок к 0. Значит, модель потенциально успешна.

### Случайный лес в классификации

In [ ]:
%%time
wood_best_model = None
wood_best_result = 0
wood_best_est = 0
wood_best_depth = 0

for est in range(1, 51):
    model_plot_v = []
    model_plot_i = []
    for depth in range (1, 31):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > wood_best_result:
            wood_best_model = model
            wood_best_result = result
            wood_best_est = est
            wood_best_depth = depth
        model_plot_v.append(result)
        model_plot_i.append(depth)
    pd.Series(model_plot_v, index=model_plot_i).plot(grid=True)
print("Accuracy наилучшей модели на валидационной выборке:", wood_best_result, "Количество деревьев:", wood_best_est, "Максимальная глубина:", depth)
plt.title('Успешность модели случайного леса в зависимости от глубины')
plt.xlabel("Глубина леса")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
model_plot_v = []
model_plot_i = []
for depth in range (1, 30):
    model = RandomForestClassifier(random_state=12345, n_estimators=wood_best_est, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    model_plot_v.append(result)
    model_plot_i.append(depth)
pd.Series(model_plot_v, index=model_plot_i).plot(grid=True)
plt.title('Успешность модели случайного леса (43) в зависимости от глубины')
plt.xlabel("Глубина леса")
plt.ylabel("Accuracy")
plt.show()

Accuracy наилучшей модели на валидационной выборке: 0.8132295719844358 Количество деревьев: 43 Максимальная глубина: 30  
Мы нашли лучшую модель случайного леса.

In [ ]:
%%time
wood_model = RandomForestClassifier(random_state=12345)
parametrs = { 'n_estimators': range (1, 51),
              'max_depth': range (1, 31)}
grid = GridSearchCV(wood_model, parametrs, cv=5)
grid.fit(features_train, target_train)
grid.best_params_
grid.best_score_

Использование GridSearchCV было эффективнее, но дольше по времени подбирались параметры, поэтому я закомментировал код, но сохранил вывод параметров. Затем воссоздал модель, далее работать буду с ней:

In [ ]:
wood_best_model = RandomForestClassifier(n_estimators=50, max_depth=8, random_state=12345) 
wood_best_model.fit(features_train, target_train) 
wood_predictions_valid = wood_best_model.predict(features_valid)
print(accuracy_score(target_valid, wood_predictions_valid ))

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(target_valid, wood_predictions_valid)).plot();

Матрица показывает, что у нас 501+125 верных прогнозов и 111+34 неверных прогнозов. 

In [ ]:
print(classification_report(target_valid, wood_predictions_valid))

Точность предсказаний для тарифа Смарт равна 87%, точность предсказаний тарифа Ультра 63%.

### Вывод

Таким образом, мы проанализировали три модели:
- логистистическая регрессия
- решающее дерево
- случайный лес
Вспомним критерии каждой модели:
<pre>
| Решающее дерево         | Низкое  | Высокая |
| Случайный лес           | Высокое | Низкая  |
| Логистическая регрессия | Среднее | Высокая |</pre>

Исходя из результатов проверки на тестовой выборке, 3 модели достигли результатов. Однако исходя из таблицы и значений, модель, которую стоит использовать - это случайный лес. Accuracy и f1-score по обоим классам у неё выше. 
Если заказчику будет нужна модель быстрее, то лучше использовать дерево решений, которое незначительно ниже.

## Проверьте модель на тестовой выборке

Переобучим модель случайного леса по train+valid данным и проверим её на тестовой выборке.

### Дерево решений

In [ ]:
wood_best_model = RandomForestClassifier(n_estimators=50, max_depth=8, random_state=12345) 
wood_best_model.fit(pd.concat([features_train, features_valid]), pd.concat([target_train, target_valid])) 
wood_predictions_valid = wood_best_model.predict(features_test)
print(accuracy_score(target_test, wood_predictions_valid ))

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(target_test, wood_predictions_valid)).plot();

Матрица показывает, что у нас 332+90 верных прогнозов и 25+68 неверных прогнозов.

In [ ]:
print(classification_report(target_test, wood_predictions_valid))

Точность предсказаний для тарифа Смарт равна 88%, точность предсказаний тарифа Ультра 66%.

Мы переобучили и ещё раз проанализировали метрики модели. Её accuracy составляет больше 0.75 (почти 0.82), следовательно, такая модель может быть принята. 

## (бонус) Проверьте модели на адекватность

In [ ]:
import numpy as np
from sklearn.dummy import DummyClassifier
X = np.array([-1, 1, 1, 1])
y = np.array([0, 1, 1, 1])
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
DummyClassifier(strategy='most_frequent')
test_model_prediction = dummy_clf.predict(features_valid)
dummy_clf.score(test_model_prediction, target_valid)

In [ ]:
import random 
random_prediction = [random.randint(0, 1) for i in range(len(test_model_prediction))]
accuracy_score(random_prediction, target_valid)

Так как рандомный массив и Dummy-модель всё-таки имеют accuracy меньше, чем наша модель, то можно признать её адекватной.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
